In [1]:
from bs4 import BeautifulSoup
import time
from selenium.webdriver import Chrome
import openpyxl
import pandas as pd
import requests
from urllib.request import urlopen
from urllib.error import URLError, HTTPError
import re
from pykospacing import spacing
from konlpy.tag import Komoran
from nltk import Text
from konlpy.tag import Komoran
from nltk import Text

In [2]:
browser = Chrome()

title = []
links = []

for i in range(1,2):
    browser.get("https://www.fmkorea.com/index.php?mid=best2&listStyle=list&page=" + str(i))
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    result = soup.find('div', {'class': 'content_dummy'}).findAll('td', class_='hotdeal_var8')
    time.sleep(3)
    
    for td in result:
        f_link = td.findAll('a', class_='hx')
        for a in f_link:
            conlink = 'https://www.fmkorea.com' + a.get('href')
            links.append(conlink)

    for i in result:
        i = BeautifulSoup(str(i), 'html.parser')
        fm_title = i.find("a", class_="hx").text
        fm_title = re.sub('[ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', '', fm_title)
        fm_title = re.sub('[\xa0|0xed]', '', fm_title)
        fm_title = re.sub('[\U0001f61f|\U0001f606]', '', fm_title)
        fm_title = re.sub('[-=+_★♥♡,#/\?:╋^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》—;]','',fm_title)
        fm_title = spacing(fm_title)
        title.append(fm_title)

In [3]:
texts = []
#본문
for i in links:
    try:
        res = requests.get(i)
        res.raise_for_status()
        res.endcoding = None
        html2 = res.text
    
        soup = BeautifulSoup(html2, 'html.parser')
        contentArea = soup.find("div", {"class" : "content_dummy"})
        p = contentArea.find("div", {"class" : "rd_body clear"})
        parags = contentArea.findAll("article")
        time.sleep(3)
        
        content = ""

        for parag in parags:
            content += parag.text
        content = re.sub('[ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', '', content)
        content = re.sub('[&nbsp;|\n|\t|\r]', '', content)
        content = re.sub('[\xa0|0xed]', '', content)
        content = re.sub('[\U0001f61f|\U0001f606]', '', content)
        content = re.sub('[-=+_★♥♡,#/\?:╋^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》—;]', '', content)
        content = spacing(content)

        texts.append(content)
   
    except HTTPError as e:
        texts.append('')
    except URLError as e:
        texts.append('')
    except AttributeError as e:
        texts.append('')

In [7]:
comments = []
#댓글
for i in links:
    try:
        res = requests.get(i)
        res.raise_for_status()
        res.endcoding = None
        html2 = res.text

        soup = BeautifulSoup(html2, 'html.parser')
        contentArea = soup.find("div", {"class" : "content_dummy"})
        parags = contentArea.findAll("ul", {"class" : "fdb_lst_ul"})
        time.sleep(3)
        
        content = ""

        for parag in parags:
            content += parag.text
        content = re.sub('[ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', '', content)
        content = re.sub('[&nbsp;|\n|\t|\r]', '', content)
        content = re.sub('[\xa0|0xed]', '', content)
        content = re.sub('[\U0001f61f|\U0001f606]', '', content)
        content = re.sub('[-=+_★♥♡,#/\?:╋^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》—;]', '', content)
        content = spacing(content)

        comments.append(content)
        
    except HTTPError as e:
        comments.append('')
    except URLError as e:
        comments.append('')
    except AttributeError as e:
        comments.append('')

In [8]:
fm_dict = {
    '제목' : title,
    '본문' : texts,
    '댓글' : comments
}

df = pd.DataFrame(fm_dict) 
df.to_csv('fm_엑셀확인용.csv', index=False, encoding="utf-8-sig")
df.to_csv('fm.csv', index=False, encoding="euc-kr")

In [ ]:
# #형태소 분석
# komoran = Komoran() 

# title_morphs = []
# txt_morphs = []
# comment_morphs = []
    
# for i in title:
#     title_morphs.append(komoran.morphs(i))
    
# for i in texts:
#     txt_morphs.append(komoran.morphs(i))
    
# for i in comments:
#     comment_morphs.append(komoran.morphs(i))  

# morphs_dict = {
#     '제목 형태소' : title_morphs,
#     '본문 형태소' : txt_morphs,
#     '댓글 형태소' : comment_morphs
# }

# df2 = pd.DataFrame(morphs_dict)
# df2.to_csv('fm_Morphs.csv', index=False, encoding="utf-8-sig")